# 📘 Introduction
This notebook covers the full pipeline for collecting, cleaning, and preparing stock + earnings event data.
The output is a cleaned dataset (`strategy_df`) used for modeling and strategy simulation.

## 📦 Data Collection
We begin by collecting three main datasets:
- `cleaned_stock_data`: Historical stock price data (daily OHLCV)
- `combined_alpha_earnings_data`: Earnings report announcements
- `ticker_df`: Metadata and fundamentals (sector, employees, CEO, etc.)

**Key Merge Strategy:**
- Merge on `Ticker` and `reported_date_clean` (earnings date)
- Dates are converted to `datetime64` for proper filtering.

## Data Preprocessing Pipeline
For each earnings event, we extract a window of ±N days.
- `Days_From_Earnings`: Offset in days from the earnings announcement
- `Next_Day_Open`: Next day's open price (used for after-hours return)
- `Regular_Change%`: Day change from Open to Close
- `After_Hours_Change%`: Change from Close to Next Day Open

EPS metrics from `earnings_df` are also injected into the window.

## 🧠 Feature Engineering & Final Columns
Only safe (non-leaky) columns are retained for modeling. These include:
- Price-based returns: `Regular_Change%`, `After_Hours_Change%`
- EPS details: `EPS_Actual`, `EPS_Estimate`, `EPS_Surprise`, `EPS_Surprise_%`
- Context: `Days_From_Earnings`, `Sector`, `Profitable`, `FemaleCEO`, etc.

All rows with missing critical values are dropped.

## 💾 Export
The result is saved as `strategy_df_window7.csv` and used in the modeling phase.

This notebook covers the full pipeline for collecting, cleaning, and preparing stock + earnings event data. The output of this notebook is a cleaned dataset called `strategy_df`, which will be used for modeling and simulation in subsequent notebooks.

Loading the kaggle auth json into the colab session.
Once you login into kaggle, this can be download from the logged in session

In [ ]:
import os
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Print the current working directory
print("\nCurrent working directory:")
print(os.getcwd())

# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# DataSet 1: Ticker Information (ticker_df):
Download the fortune 1000 company information from Kaggle


In [ ]:
import kaggle
import os

dataset_name = 'jeannicolasduval/2024-fortune-1000-companies'

kaggle.api.dataset_download_files(dataset_name, path='.', unzip=True)

download_path = 'ticker/'
if not os.path.exists(download_path):
    os.makedirs(download_path)
kaggle.api.dataset_download_files(dataset_name, path=download_path, unzip=True)


Loading the data into a data frame ticker_df.

In [ ]:
import pandas as pd
import os


file_path = './ticker/fortune1000_2024.csv'
if os.path.exists(file_path):
    try:
        ticker_df = pd.read_csv(file_path, index_col=0)
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found. Double-check the path.")
    except pd.errors.EmptyDataError:
        print(f"Error: File '{file_path}' is empty.")
    except pd.errors.ParserError:
        print(f"Error: Problem parsing '{file_path}'. Check for incorrect delimiters or data types.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
else:
   print(f"Error: File '{file_path}' not found. Make sure the filename and path are correct.")
   ticker_df = None


ticker_df.head()


# DataSet 2: Get earning report dates from Alpha Vantage (combined_alpha_earnings_data):
Invoking the alpha vantage earnings API

In [ ]:
import requests
import pandas as pd

api_key = "8Y236L6QW3EKZD6K"

def get_earnings_data(ticker):
    """Fetch earnings report dates from Alpha Vantage"""
    url = f"https://www.alphavantage.co/query?function=EARNINGS&symbol={ticker}&apikey={api_key}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        data = response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {ticker}: {e}")
        return pd.DataFrame()  # Return an empty DataFrame on error

    earnings_list = []

    if "quarterlyEarnings" in data:
        for entry in data["quarterlyEarnings"]:
            try:
                earnings_list.append({
                    "symbol": ticker,
                    "reported_date": pd.to_datetime(entry["reportedDate"]),
                    "actual_eps": float(entry["reportedEPS"]) if entry["reportedEPS"] != "None" else None,
                    "estimated_eps": float(entry["estimatedEPS"]) if entry["estimatedEPS"] != "None" else None,
                    "surprise": float(entry["surprise"]) if entry["surprise"] != "None" else None,
                    "surprise_pct": float(entry["surprisePercentage"]) if entry["surprisePercentage"] != "None" else None,
                })
            except KeyError as e:
                print(f"KeyError in earnings data for {ticker}: {e}")
                continue

    print(f"Loaded {len(earnings_list)} earnings reports for {ticker}")

    return pd.DataFrame(earnings_list)

Since the API was rate-limited. Pulled the data in batches

In [ ]:
# Function to gather earnings data for all tickers using your existing function
def get_all_earnings_data(tickers, get_earnings_data_fn):
    all_earnings = []
    for ticker in tickers:
        try:
            df = get_earnings_data_fn(ticker)
            if not df.empty:
                all_earnings.append(df)
                print(f"Fetched earnings for {ticker}")
        except Exception as e:
            print(f"Failed for {ticker}: {e}")
    return pd.concat(all_earnings, ignore_index=True)

# Run this for a small subset first to test (e.g., first 10 tickers)
earnings_data_top10 = get_all_earnings_data(top_10_tickers, get_earnings_data)
earnings_data_top10.head()

In [ ]:
api_key = "8Y236L6QW3EKZD6K"

# These are the 24 tickers still needed (after removing the Top 10 already fetched)
tickers_missing = [
    'PSX', 'CI', 'MPC', 'HD', 'GOOG', 'XOM', 'MCK', 'CNC', 'GM', 'CVX', 'C',
    'UNH', 'BRK.A', 'F', 'CVS', 'BAC', 'COST', 'WBA', 'FB', 'KR', 'CAH',
    'ABC', 'WMT', 'ELV', 'VLO'
]

# Fetch only these (add delay=13 to avoid rate limit per minute)
earnings_data_remaining = get_all_earnings_data(tickers_missing, get_earnings_data, delay=13)

# Combine with the existing top 10 data
earnings_data_top30 = pd.concat([earnings_data_top10, earnings_data_remaining], ignore_index=True)

# Check result
print("✅ Total earnings entries (Top 30):", len(earnings_data_top30))
earnings_data_top30['symbol'].nunique()

In [ ]:
tickers_to_retry = ['META', 'WMT', 'ELV']

earnings_retry = get_all_earnings_data(tickers_to_retry, get_earnings_data, delay=13)

earnings_data_top30 = pd.concat([earnings_data_top30, earnings_retry], ignore_index=True)

In [ ]:
# Combine both earnings datasets into one
earnings_data_full = pd.concat([earnings_data_top30, earnings_data_remaining], ignore_index=True)

# Optional: Drop duplicates just in case (some overlaps could exist)
earnings_data_full.drop_duplicates(subset=['symbol', 'reported_date'], inplace=True)

In [ ]:
# Convert event-based top30 earnings to the same format
alpha_format_top30 = earnings_data_top30[[
    'Ticker', 'Earnings_Date', 'EPS_Actual', 'EPS_Estimate', 'EPS_Surprise', 'EPS_Surprise_%'
]].drop_duplicates()

# Rename columns to match the earnings_data_remaining format
alpha_format_top30 = alpha_format_top30.rename(columns={
    'Ticker': 'symbol',
    'Earnings_Date': 'reported_date',
    'EPS_Actual': 'actual_eps',
    'EPS_Estimate': 'estimated_eps',
    'EPS_Surprise': 'surprise',
    'EPS_Surprise_%': 'surprise_pct'
})

In [ ]:
# Combine the two
all_earnings_alpha = pd.concat([alpha_format_top30, earnings_data_remaining], ignore_index=True)

# Drop any exact duplicates (some may overlap)
all_earnings_alpha = all_earnings_alpha.drop_duplicates(subset=['symbol', 'reported_date'])

# Check how many unique tickers we now have
print("🧠 Total earnings records:", len(all_earnings_alpha))
print("🔢 Unique tickers with Alpha Vantage data:", all_earnings_alpha['symbol'].nunique())
print("🧾 Example tickers:", all_earnings_alpha['symbol'].unique()[:10])

In [ ]:
Taking a back up to ensure the data can be reloaded on a different day

In [ ]:
# STEP 1: Save the combined earnings data to a CSV
all_earnings_alpha.to_csv("combined_alpha_earnings_data.csv", index=False)

# STEP 2: Create a download link (works in Colab)
from google.colab import files
files.download("combined_alpha_earnings_data.csv")

Reload the data on a different day

In [ ]:
import pandas as pd

# Adjust the path based on where your file is stored in Drive
file_path = '/content/combined_alpha_earnings_data (1).csv'
combined_alpha_earnings_data = pd.read_csv(file_path)

# Preview
combined_alpha_earnings_data.head()

# DataSet 3: Stock Daily trade Information (combined_stock_data):
Bulk data set of stock ticker information until december 2022

In [ ]:
import kaggle
import os

dataset_name = 'paultimothymooney/stock-market-data'

kaggle.api.dataset_download_files(dataset_name, path='.', unzip=True)

download_path = 'data/'
if not os.path.exists(download_path):
    os.makedirs(download_path)
kaggle.api.dataset_download_files(dataset_name, path=download_path, unzip=True)


Only loading tickers present in ticker_df in order to simplify the bulk data set that is present in this kaggle set

In [ ]:
import os
import numpy as np
import pandas as pd

def add_ticker_column(df, filename, index):
    """Adds a ticker column to the DataFrame based on the filename."""
    ticker = filename.split('.')[0]  # Extract ticker from filename
    df['Ticker'] = ticker
    df['Index'] = index
    return df

def load_and_combine_csv_data(base_dir, ticker_df):
    """
    Loads CSV files only for tickers present in ticker_df, adds a Ticker column,
    and combines them into a single DataFrame.
    """
    if ticker_df is None:
        print("Error: ticker_df is None.  Ensure it's loaded before calling this function.")
        return None

    valid_tickers = ticker_df['Ticker'].str.upper().tolist()  # Get list of valid tickers, convert to uppercase

    all_data = []
    for subdir in os.listdir(base_dir): # go through base, where stock market dir is
        subdir_path = os.path.join(base_dir, subdir)
        if os.path.isdir(subdir_path): # If is dir

            csv_dir = os.path.join(subdir_path, "csv") # go to the csv dir inside the base
            if os.path.exists(csv_dir):
                for filename in os.listdir(csv_dir):
                    if filename.endswith(".csv"):
                        ticker = filename.split('.')[0].upper()  # Extract ticker and convert to uppercase

                        #if ticker in valid_tickers:  # Check if ticker is in the valid list
                        csv_path = os.path.join(csv_dir, filename)
                        try:
                            df = pd.read_csv(csv_path, parse_dates=['Date'])  # Parse date

                            # Date is not defined, print so we know
                            if 'Date' not in df:
                                print(f"Issue, Date column not defined in {filename}")

                            df = add_ticker_column(df, filename, subdir_path)  # ticker to df from file name
                            all_data.append(df)  # Append the df to all_data

                        except Exception as e:
                            print(f"Error loading or processing {filename}: {e}")
                        # else:
                        #     print(f"Skipping {filename}: Ticker not found in ticker_df.")

            else:
                print(f"csv directory not found in {subdir}")

    if not all_data:
        print("No data loaded. Ensure the base directory contains the specified tickers, and the CSV files are correctly named.")
        return None

    combined_df = pd.concat(all_data, ignore_index=True)  # combine
    return combined_df

# --- Main ---
base_dir = "/content/stock_market_data/"  # The main directory with the four folders

# ASSUMPTION: ticker_df is already loaded and accessible in this scope!

combined_stock_data = load_and_combine_csv_data(base_dir, ticker_df)  # Loads data for tickers present in ticker_df.  Pass ticker_df to the function

if combined_stock_data is not None:
    print(f"Combined data shape: {combined_stock_data.shape}")
    print(f"Combined data columns: {combined_stock_data.columns}")
    print(combined_stock_data.head())  # Print the head of the data
else:
    print("Failed to load combined stock data.")

In [ ]:
def inspect_and_clean_ticker(df, ticker):
    """Clean and return stock data for a single ticker."""
    subset = df[df['Ticker'] == ticker].copy()
    subset['Date'] = pd.to_datetime(subset['Date'], errors='coerce')
    cleaned = subset.dropna(subset=['Date'])

    if cleaned.empty:
        print(f"⚠️ No clean data found for {ticker}")
    else:
        print(f"✅ {ticker}: {cleaned.shape[0]} rows | Date range: {cleaned['Date'].min().date()} → {cleaned['Date'].max().date()}")

    return cleaned

# ✅ Loop over all tickers present in the earnings dataset
tickers_to_clean = all_earnings_alpha['symbol'].unique().tolist()
cleaned_ticker_dfs = []

for ticker in tickers_to_clean:
    cleaned = inspect_and_clean_ticker(filtered_stock_df, ticker)
    if not cleaned.empty:
        cleaned_ticker_dfs.append(cleaned)

# 📦 Combine all cleaned ticker data
cleaned_stock_data = pd.concat(cleaned_ticker_dfs, ignore_index=True)

print(f"\n📊 Final cleaned stock data shape: {cleaned_stock_data.shape}")
print(f"🔢 Tickers included: {cleaned_stock_data['Ticker'].nunique()}")


Preparing a final combined dataset required for processing for a specific window size this method can be run multiple times and used to extract the data.

In [ ]:
import pandas as pd
import numpy as np

def build_event_and_strategy_df(stock_df, earnings_df, ticker_df, window=5):
    # Step 1: Clean & Convert Dates
    stock_df["Date"] = pd.to_datetime(stock_df["Date"])
    earnings_df["reported_date_clean"] = pd.to_datetime(earnings_df["reported_date_clean"])

    # Step 2: Sort for merge
    stock_df = stock_df.sort_values(["Ticker", "Date"])

    # Step 3: Build event windows
    event_rows = []
    for (ticker, reported_date) in earnings_df[["symbol", "reported_date_clean"]].dropna().values:
        price_df = stock_df[stock_df["Ticker"] == ticker]
        idx = price_df[price_df["Date"] == reported_date].index
        if len(idx) == 0:
            continue
        loc = price_df.index.get_loc(idx[0])
        start = max(0, loc - window)
        end = min(len(price_df), loc + window + 1)
        window_df = price_df.iloc[start:end].copy()
        window_df["Days_From_Earnings"] = np.arange(start - loc, end - loc)
        window_df["Earnings_Date"] = reported_date
        window_df["symbol"] = ticker
        # Merge EPS info
        for col in ['actual_eps', 'estimated_eps', 'surprise', 'surprise_pct']:
            window_df[col] = earnings_df[(earnings_df["symbol"] == ticker) &
                                         (earnings_df["reported_date_clean"] == reported_date)][col].values[0]
        event_rows.append(window_df)

    event_df = pd.concat(event_rows).reset_index(drop=True)

    # Step 4: Add Next Day Open
    event_df["Next_Day_Open"] = event_df.groupby("Ticker")["Open"].shift(-1)

    # Step 5: Add Return Metrics
    event_df["Regular_Change%"] = 100 * (event_df["Close"] - event_df["Open"]) / event_df["Open"]
    event_df["After_Hours_Change%"] = 100 * (event_df["Next_Day_Open"] - event_df["Close"]) / event_df["Close"]

    # Step 6: Merge Ticker Metadata
    enriched_df = pd.merge(event_df, ticker_df, how="left", left_on="Ticker", right_on="Ticker")

    # Rename EPS fields
    enriched_df = enriched_df.rename(columns={
        "surprise": "EPS_Surprise",
        "surprise_pct": "EPS_Surprise_%",
        "actual_eps": "EPS_Actual",
        "estimated_eps": "EPS_Estimate"
    })

    # Reorder for modeling
    strategy_cols = [
        'Ticker', 'Earnings_Date', 'Date', 'Days_From_Earnings',
        'Open', 'Close', 'Next_Day_Open',
        'Regular_Change%', 'After_Hours_Change%', 'EPS_Actual', 'EPS_Estimate',
        'EPS_Surprise', 'EPS_Surprise_%',
        'Sector', 'Industry', 'Profitable', 'Founder_is_CEO',
        'FemaleCEO', 'Growth_in_Jobs'
    ]
    strategy_df = enriched_df[strategy_cols].dropna()

    return strategy_df


In [ ]:
strategy_df_1 = build_event_and_strategy_df(cleaned_stock_data, combined_alpha_earnings_data, ticker_df, window=1)
strategy_df_1.to_csv("/content/strategy_df_window1.csv", index=False)
strategy_df_1.head()

In [ ]:
strategy_df_3 = build_event_and_strategy_df(cleaned_stock_data, combined_alpha_earnings_data, ticker_df, window=3)
strategy_df_3.to_csv("/content/strategy_df_window3.csv", index=False)
strategy_df_3.head()

In [ ]:

strategy_df_5 = build_event_and_strategy_df(cleaned_stock_data, combined_alpha_earnings_data, ticker_df, window=5)
strategy_df_5.to_csv("/content/strategy_df_window5.csv", index=False)
strategy_df_5.head()

In [ ]:
strategy_df_7 = build_event_and_strategy_df(cleaned_stock_data, combined_alpha_earnings_data, ticker_df, window=7)
strategy_df_7.to_csv("/content/strategy_df_window7.csv", index=False)
strategy_df_7.head()

In [ ]:
strategy_df_9 = build_event_and_strategy_df(cleaned_stock_data, combined_alpha_earnings_data, ticker_df, window=9)
strategy_df_9.to_csv("/content/strategy_df_window9.csv", index=False)
strategy_df_9.head()